# NN Training

This notebook was used to train multiple neural networks with different hyper-parameters in order to find the best one.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
from tensorflow import keras

## Pre-processing

Load and pre-process the dataset:

In [4]:
from lib import pre_processing
file_path = "./data/features_3_sec.csv"
df = pd.read_csv(file_path)
df = pre_processing(df)

## Data Normalization

In [5]:
from lib import normalize_df
df = normalize_df(df)

In [6]:
df_numpy = df.to_numpy()
X = df_numpy[:,:-1].astype(np.float32)
y = df_numpy[:, -1]

In [7]:
X.shape, y.shape

((9990, 57), (9990,))

## Label pre-processing

In [8]:
from lib import encode_labels
y_ohe = encode_labels(y)

In [9]:
X.shape, y_ohe.shape

((9990, 57), (9990, 10))

# Model selection

Given the complexity of the models, we will select the best model using a simple train-validation split, without resorting to cross-validation.

In [10]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=25,
        verbose=0,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1,
        patience=5,
        verbose=0,
        min_lr=1e-7
    ),
    keras.callbacks.TerminateOnNaN()
]

## Multiclass classifier

In [11]:
# Connect to the database
from pymongo import MongoClient
client = MongoClient('localhost', 27018)
db = client['nn_training']
collection = db['musical_genre_multiclass']

In [12]:
from sklearn.model_selection import train_test_split

def train_multiclass(X, y, test_size, num_hidden_layers, num_neurons, activation, epochs, batch_size, verbose, use_db=True):
    # Document structure
    json = {
        "num_hidden_layers": int(num_hidden_layers),
        "num_neurons": int(num_neurons),
        "activation_f": activation,
        "batch_size": int(batch_size)
    }

    if use_db is True:
        # Check into the DB if this configuration was already trained
        if not collection.count_documents(json) == 0:
            return
    
    # Input layer
    layers = [keras.layers.Dense(num_neurons, activation=activation, input_shape=(57,))]
    
    # Hidden layer
    for i in range(num_hidden_layers):
        layers.append(keras.layers.Dense(num_neurons, activation=activation))
        
    # Output layers
    layers.append(keras.layers.Dense(10))
    layers.append(keras.layers.Softmax())
      
    # Create the model
    model = keras.Sequential(layers)
        
    # Train
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=1e-3),
        loss = keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, validation_data = (X_test, y_test), batch_size=batch_size, callbacks=callbacks, verbose=verbose)
    
    
    # Log the result to the DB
    if use_db is True:
        json["validation_accuracy"] = float(model.get_metrics_result()['accuracy'])
        collection.insert_one(json)
   
    # Return the trained model
    print("-----------------------------------------------------------------------------------")
    print(f"Multiclass - Validation accuracy: {model.get_metrics_result()['accuracy']:.5f}")
    return model

Training loop:

In [13]:
hidden_layers_values = np.arange(2, 4, 1)
num_neurons_values = np.arange(50, 101, 5)
batch_size_values = (5, 10, 15, 20, 30, 50)

In [14]:
from tqdm.contrib.itertools import product

for hidden_layers, num_neurons, batch_size in product(hidden_layers_values, num_neurons_values, batch_size_values):
    model = train_multiclass(
        X=X,
        y=y_ohe,
        test_size=0.2,
        num_hidden_layers=hidden_layers,
        num_neurons=num_neurons,
        activation='selu',
        epochs=150, 
        batch_size=batch_size,
        verbose=0
    )

  0%|          | 0/132 [00:00<?, ?it/s]

## Balanced binary classifiers

In [15]:
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [16]:
# Connect to the database
client = MongoClient('localhost', 27018)
db = client['nn_training']
collection = db['musical_genre_binary']

In [17]:
def train_binary(X, y, test_size, num_hidden_layers, num_neurons, activation, epochs, batch_size, verbose, use_db=True):
     # Document structure
    json = {
        "num_hidden_layers": int(num_hidden_layers),
        "num_neurons": int(num_neurons),
        "activation_f": activation,
        "batch_size": int(batch_size),
        "genre": genre
    }
    if use_db is True:


        # Check into the DB if this configuration was already trained
        if not collection.count_documents(json) == 0:
            return

    # Input layer
    layers = [keras.layers.Dense(num_neurons, activation=activation, input_shape=(57,))]
    
    # Hidden layers
    for i in range(num_hidden_layers):
        layers.append(keras.layers.Dense(num_neurons, activation=activation))
        
    # Output layers
    layers.append(keras.layers.Dense(1, activation='sigmoid'))
      
    # Create the model
    model = keras.Sequential(layers)

    # Train
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=1e-3),
        loss = keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, validation_data = (X_test, y_test), batch_size=batch_size, callbacks=callbacks, verbose=verbose)
    
    # Log the result to the DB
    if use_db is True:
        json["validation_accuracy"] = float(model.get_metrics_result()['accuracy'])
        collection.insert_one(json)
       
    # Return the trained model
    print("-----------------------------------------------------------------------------------")
    print(f"Genre: {genre} - Validation accuracy: {model.get_metrics_result()['accuracy']:.5f}")
    return model

Training loop:

In [18]:
hidden_layers_values = np.arange(1, 3, 1)
num_neurons_values = np.arange(15, 101, 5)
batch_size_values = (5, 10, 15, 20, 30, 50)

In [19]:
from lib import balance_classes
from tqdm.notebook import tqdm

for genre in tqdm(genres):
    Xi, yi = balance_classes(X, y, genre)
    for hidden_layers, num_neurons, batch_size in product(hidden_layers_values, num_neurons_values, batch_size_values):
        model = train_binary(
            X=Xi,
            y=yi,
            test_size=0.2,
            num_hidden_layers=hidden_layers,
            num_neurons=num_neurons,
            activation='selu', 
            epochs=150, 
            batch_size=batch_size,
            verbose=0
        )

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

...

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

# Export the models

The hyper-parameters of the best models are retrieved from the database through the id of the corresponding document.

We will save the topology and the hyper-parameters of those models to use them in another notebook.

Note that we will not train those models, that is in order to easily restore the uninitialized weights if needed.

## Multiclass classifier

In [20]:
# Connect to the database
client = MongoClient('localhost', 27018)
db = client['nn_training']
collection = db['musical_genre_multiclass']

In [21]:
from bson.objectid import ObjectId
hyper_params = collection.find_one({"_id": ObjectId('63ff5d4f321be25ff4bd4a4a')})

In [22]:
# Input layer
layers = [keras.layers.Dense(hyper_params['num_neurons'], activation=hyper_params['activation_f'], input_shape=(57,))]

# Hidden layer
for i in range(hyper_params['num_hidden_layers']):
    layers.append(keras.layers.Dense(hyper_params['num_neurons'], activation=hyper_params['activation_f']))

# Output layers
layers.append(keras.layers.Dense(10))
layers.append(keras.layers.Softmax())

# Create and save the model
model = keras.Sequential(layers)
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=1e-3),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])
model.save('models/multiclass', save_format='h5')

## Balanced binary classifiers

In [23]:
# Connect to the database
client = MongoClient('localhost', 27018)
db = client['nn_training']
collection = db['musical_genre_binary']

In [24]:
best_models = {
    'blues':     '63f770df24aa2c8c48d8d6be',
    'classical': '63f78119d5a599ee96990375',
    'country':   '63f794e6bddddbe6626012fe',
    'disco':     '63f5ff930ba565f9849e71f1',
    'hiphop':    '63f4da42fda8e4d8c6aaa53e',
    'jazz':      '63f50842a7c97cc804df9978',
    'metal':     '63f51f4b9298490f118ba650',
    'pop':       '63ffbfd3305fa7380dc8c275',
    'reggae':    '63f540751a4de2a08e300398',
    'rock':      '63f5524b3541477d0005d66e'
}

In [25]:
for genre in genres:
    hyper_params = collection.find_one({"_id": ObjectId(best_models[genre])})
    
    # Input layer
    layers = [keras.layers.Dense(hyper_params['num_neurons'], activation=hyper_params['activation_f'], input_shape=(57,))]

    # Hidden layers
    for i in range(hyper_params['num_hidden_layers']):
        layers.append(keras.layers.Dense(hyper_params['num_neurons'], activation=hyper_params['activation_f']))

    # Output layers
    layers.append(keras.layers.Dense(1, activation='sigmoid'))

    # Create and save the model
    model = keras.Sequential(layers)
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=1e-3),
        loss = keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'])
    model.save('models/' + genre, save_format='h5')